需求：

北京明天的天气怎么样？上海呢？ （通过两次对话实现）

以ReAct模式为例进行实现：



In [5]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")


# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=2)

# 获取一个搜索的工具
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

# 获取一个记忆的实例：ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"  #此时必须设置memory_key,且值为chat_history。因为提示词模板中使用的是chat_history。二者必须一致！
)

# 获取Agent的实例
agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION

# 获取AgentExecutor的实例
agent_executor = initialize_agent(
    tools = [search_tool],
    llm = llm,
    agent = agent,
    verbose = True, # 显示详细的日志信息
    memory = memory,
)


# 通过AgentExecutor 调用invoke(),并得到响应
result = agent_executor.invoke("查询lol全球总决赛谁打谁今天是2025.10.30")


# 处理响应数据
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "LOL 全球总决赛 2025.10.30 对阵"
```
Observation: [{'title': '2025全球总决赛入围赛及瑞士轮赛程公布！ - 英雄联盟', 'url': 'https://lol.qq.com/news/detail.shtml?&docid=10496120635862816173', 'content': '2025全球总决赛入围赛及瑞士轮赛程公布！\n\n2025英雄联盟全球总决赛将于北京时间10月14日正式开赛，入围赛LPL和LCK的四号种子将进行一场Bo5对决，胜者将晋级瑞士轮阶段。\n\n四海聚此役，争者留其名！ 来自全球各大赛区的顶尖强队已集结完毕：\n\nLPL赛区：BLG、AL、TES、IG\n\nLCK赛区：GEN、HLE、KT、T1\n\nLCP赛区：CFO、TSW、PSG\n\nLTA赛区：FLY、VKS、100T\n\nLEC赛区：G2、MKOI、FNC\n\n群雄汇聚中国，巅峰对决，敬请期待！\n\n入围赛阶段：10月14日；瑞士轮阶段：10月15日-10月19日、10月22日-10月25日；淘汰赛阶段：10月28日-11月2日；冠亚军决赛：11月9日。 让我们共同期待，谁将问鼎巅峰！\n\nImage 9\n\nImage 10\n\n0)\n\nImage 11\n\n英雄联盟赛事官方\n8621.1万次 总阅读量\n\n38.3万粉丝\n\n关注\n\nTA的文章\n\n   Image 12 2025全球总决赛淘汰赛1/4决赛第二日TOP5\n\n427阅读 \n   Image 13 拜仁慕尼黑为LPL加油助威！', 'score': 0.7211353}, {'title': '英雄聯盟2025賽季世界大賽 - 維基百科', 'url': 'https://zh.wikipedia.org/wiki/%E8%8B%B1%E9%9B%84%E8%81%94%E7%9B%9F2025%E8%B5%9B%E5%AD%A3%E5%85%A8%E7%90%83%E6%80%BB%E5%8

In [3]:
result = agent_executor.invoke("明天呢？")
# 处理响应数据
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: 2025年10月30日的对阵情况尚未公布，因为比赛的具体安排通常在赛事临近时才会确定。请关注官方渠道以获取最新信息。明天的比赛安排可能会在比赛前夕公布。
```

> Finished chain.
{'input': '明天呢？', 'chat_history': [HumanMessage(content='查询2025.10.30 lol全球总决赛谁打谁', additional_kwargs={}, response_metadata={}), AIMessage(content='2025年《英雄联盟》全球总决赛将于2025年10月14日至11月9日举行。总决赛定于2025年11月9日进行，比赛场馆为成都东安湖体育公园多功能体育馆。关于2025年10月30日的具体对阵情况，目前尚未公布，因为四分之一决赛的对阵将根据抽签结果确定。', additional_kwargs={}, response_metadata={}), HumanMessage(content='明天呢？', additional_kwargs={}, response_metadata={}), AIMessage(content='2025年10月30日的对阵情况尚未公布，因为比赛的具体安排通常在赛事临近时才会确定。请关注官方渠道以获取最新信息。明天的比赛安排可能会在比赛前夕公布。\n```', additional_kwargs={}, response_metadata={})], 'output': '2025年10月30日的对阵情况尚未公布，因为比赛的具体安排通常在赛事临近时才会确定。请关注官方渠道以获取最新信息。明天的比赛安排可能会在比赛前夕公布。\n```'}
